In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns


train = pd.read_csv("../input/ventilator-pressure-prediction/train.csv")
test = pd.read_csv("../input/ventilator-pressure-prediction/test.csv")

train["seq"] = train.groupby(["breath_id"])["id"].cumcount()
test["seq"] = test.groupby(["breath_id"])["id"].cumcount()


In [ ]:
train["pressure"] = np.around (train["pressure"].values, decimals=4)
             
base = train.drop_duplicates (subset=["R","C","pressure"])[["R","C", "pressure"]]
sns.set(font_scale=1.2)
for seq in range(80):
    train_seq = train.query("seq == @seq")
    
    time = train_seq["time_step"].mean()
    
    df = train_seq.groupby (["R","C","pressure"]).agg({"id":"count"}).reset_index()
    df = df.append (base)
    df = df.groupby (["R","C","pressure"]).agg({"id":"sum"}).reset_index()
    
    df["log1p_id"] = np.log1p(df["id"])

    heat_map = df.pivot(index=["R","C"], columns="pressure", values="log1p_id").fillna(0)

    fig = plt.figure ( figsize=(15,5) )
    g=sns.heatmap(heat_map, vmin=0, vmax=7, xticklabels=30, cbar_kws={'label': 'log1p(count)'})


    plt.text(670.0, 0.7, f"frame: {seq}, time: {time:.2f}", horizontalalignment='left', size='large', color='white', weight='semibold')
    plt.tight_layout()
    plt.savefig(f'frame{seq:05d}.jpg')   
    if seq == 0:
        plt.show()
    plt.close(fig)


In [ ]:
!ffmpeg -y -framerate 5  -i ./frame%5d.jpg  output.mp4

In [ ]:
from IPython.display import Video

Video("output.mp4", embed=True)